In [1]:
#Run First
import requests
from io import StringIO
import pandas as pd
import time
from bs4 import BeautifulSoup

In [2]:
standings_url = "https://www.basketball-reference.com/leagues/NBA_2024_standings.html"

In [3]:
data = requests.get(standings_url)

In [13]:
from bs4 import BeautifulSoup

In [4]:
soup = BeautifulSoup(data.text)

In [9]:
standings_table_west = soup.select('table.stats_table')[1]
standings_table_east = soup.select('table.stats_table')[0]

[<a href="/teams/OKC/2024.html">Oklahoma City Thunder</a>,
 <a href="/teams/DEN/2024.html">Denver Nuggets</a>,
 <a href="/teams/MIN/2024.html">Minnesota Timberwolves</a>,
 <a href="/teams/LAC/2024.html">Los Angeles Clippers</a>,
 <a href="/teams/DAL/2024.html">Dallas Mavericks</a>,
 <a href="/teams/PHO/2024.html">Phoenix Suns</a>,
 <a href="/teams/NOP/2024.html">New Orleans Pelicans</a>,
 <a href="/teams/LAL/2024.html">Los Angeles Lakers</a>,
 <a href="/teams/SAC/2024.html">Sacramento Kings</a>,
 <a href="/teams/GSW/2024.html">Golden State Warriors</a>,
 <a href="/teams/HOU/2024.html">Houston Rockets</a>,
 <a href="/teams/UTA/2024.html">Utah Jazz</a>,
 <a href="/teams/MEM/2024.html">Memphis Grizzlies</a>,
 <a href="/teams/SAS/2024.html">San Antonio Spurs</a>,
 <a href="/teams/POR/2024.html">Portland Trail Blazers</a>]

In [6]:
links = standings_table_east.find_all('a') + standings_table_west.find_all('a')

In [117]:
links = [l.get("href") for l in links]

In [118]:
links = [l for l in links if '/teams' in l]

In [119]:
team_urls = [f"https://www.basketball-reference.com{l}" for l in links]

In [120]:
team_url = team_urls[1]

In [121]:
data = requests.get(team_url)

In [122]:
roster = pd.read_html(data.text, match="Roster Table")[0]
stats = pd.read_html(data.text, match="Per Game Table")[0]

C:\Users\khull\AppData\Local\Temp\ipykernel_42460\1004684286.py:1: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  roster = pd.read_html(data.text, match="Roster Table")[0]


ValueError: No tables found

In [58]:
soup = BeautifulSoup(data.text)

In [59]:
links = soup.find_all('a')

In [60]:
links = [l.get("href") for l in links]

In [61]:
links = [l for l in links if l and '2024_games' in l]

In [62]:
data = requests.get(f"https://www.basketball-reference.com{links[0]}")

In [63]:
matches = pd.read_html(data.text, match="Regular Season")[0]

C:\Users\khull\AppData\Local\Temp\ipykernel_42460\3283715952.py:1: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  matches = pd.read_html(data.text, match="Regular Season")[0]


In [66]:
matches = matches.drop(columns=['Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5', 'Unnamed: 8', 'Notes'])

In [67]:
matches = matches.rename(columns={'Unnamed: 7': 'Result'}) 

In [94]:
player_stats = pd.merge(stats, roster[["Player", "No.", "Ht", "Wt", "Birth Date", "Birth", "Exp", "College"]], on = "Player", how = "inner")

In [2]:
#Run Second
all_matches = []
all_players = []
standings_url = "https://www.basketball-reference.com/leagues/NBA_2025_standings.html"
years = list(range(2024, 2020, -1))
years = [2024, 2023]

In [3]:
#Run Third
for year in years:
    data = requests.get(standings_url)
    soup = BeautifulSoup(data.text)
    standings_table_west = soup.select('table.stats_table')[1]
    standings_table_east = soup.select('table.stats_table')[0]
    links = standings_table_east.find_all('a') + standings_table_west.find_all('a')
    links = [l.get("href") for l in links]
    links = [l for l in links if '/teams' in l]
    team_urls = [f"https://www.basketball-reference.com{l}" for l in links]

    previous_season = soup.select("a.prev")[0].get("href")
    standings_url = f"https://www.basketball-reference.com{previous_season}"
    
    for team_url in team_urls:
        team_name = team_url.split("/")[-2]

        data = requests.get(team_url)
        html_data = StringIO(data.text)
        roster = pd.read_html(html_data, match="Roster Table")[0]
        stats = pd.read_html(html_data, match="Per Game Table")[0]
        player_stats = pd.merge(stats, roster[["Player", "No.", "Ht", "Wt", "Birth Date", "Birth", "Exp", "College"]], on = "Player", how = "inner")

        soup = BeautifulSoup(data.text)
        links = soup.find_all('a')
        links = [l.get("href") for l in links]
        links = [l for l in links if l and '2025_games' in l]
        data = requests.get(f"https://www.basketball-reference.com{links[0]}")
        html_data = StringIO(data.text)
        matches = pd.read_html(html_data, match="Regular Season")[0]
        matches = matches.drop(columns=['Unnamed: 3', 'Unnamed: 4', 'Unnamed: 8'])
        matches = matches.rename(columns={'Unnamed: 7': 'Result'}) 
        matches = matches.rename(columns={'Unnamed: 5': 'Venue'})
        matches['Venue'] = matches['Venue'].apply(lambda x: 'Home' if pd.isna(x) else 'Away')
        matches['Date'] = pd.to_datetime(matches['Date'], format='%a, %b %d, %Y', errors='coerce')
        matches = matches[~(
            (matches['Date'] >= '2024-12-10') & 
            (matches['Notes'].str.contains("Play-In", na=False))
        )]
        matches = matches.drop(columns=['Notes'])
        
        matches["Season"] = year
        matches["Team"] = team_name
        player_stats["Season"] = year
        player_stats["Team"] = team_name
        all_matches.append(matches)
        all_players.append(player_stats)
        time.sleep(12)

ImportError: Missing optional dependency 'html5lib'.  Use pip or conda to install html5lib.

In [ ]:
#Run Fourth
match_df = pd.concat(all_matches)
match_df = match_df[match_df["G"] != "G"]
match_df.to_csv("matches.csv", index = False)
roster_df = pd.concat(all_players)
roster_df.to_csv("players.csv")

In [8]:
#Testing for Data Manipulation
team_url = "https://www.basketball-reference.com/teams/MIL/2025.html"
data = requests.get(team_url)
html_data = StringIO(data.text)
roster = pd.read_html(html_data, match="Roster Table")[0]
stats = pd.read_html(html_data, match="Per Game Table")[0]
player_stats = pd.merge(stats, roster[["Player", "No.", "Ht", "Wt", "Birth Date", "Birth", "Exp", "College"]], on = "Player", how = "inner")

soup = BeautifulSoup(data.text)
links = soup.find_all('a')
links = [l.get("href") for l in links]
links = [l for l in links if l and '2025_games' in l]
data = requests.get(f"https://www.basketball-reference.com{links[0]}")
html_data = StringIO(data.text)
matches = pd.read_html(html_data, match="Regular Season")[0]
matches = matches.drop(columns=['Unnamed: 3', 'Unnamed: 4', 'Unnamed: 8'])
matches = matches.rename(columns={'Unnamed: 7': 'Result'}) 
matches = matches.rename(columns={'Unnamed: 5': 'Venue'})
matches['Venue'] = matches['Venue'].apply(lambda x: 'Home' if pd.isna(x) else 'Away')
matches['Date'] = pd.to_datetime(matches['Date'], format='%a, %b %d, %Y', errors='coerce')
matches = matches[~(
    (matches['Date'] >= '2024-12-10') & 
    (matches['Notes'].str.contains("In-Season", na=False))
)]
matches.head(40)

,G,Date,Start (ET),Venue,Opponent,Result,Tm,Opp,W,L,Streak,Attend.,LOG,Notes
0,1,2024-10-23,7:30p,Away,Philadelphia 76ers,W,124,109,1,0,W 1,19754,2:28,NaN
1,2,2024-10-25,8:00p,Home,Chicago Bulls,L,122,133,1,1,L 1,17790,2:13,NaN
2,3,2024-10-27,6:00p,Away,Brooklyn Nets,L,102,115,1,2,L 2,17926,2:19,NaN
3,4,2024-10-28,7:30p,Away,Boston Celtics,L,108,119,1,3,L 3,19156,2:18,NaN
4,5,2024-10-31,8:00p,Away,Memphis Grizzlies,L,99,122,1,4,L 4,15377,2:12,NaN
5,6,2024-11-02,8:00p,Home,Cleveland Cavaliers,L,113,114,1,5,L 5,17341,2:15,NaN
6,7,2024-11-04,7:00p,Away,Cleveland Cavaliers,L,114,116,1,6,L 6,19432,2:02,NaN
7,8,2024-11-07,8:00p,Home,Utah Jazz,W,123,100,2,6,W 1,17341,2:14,NaN
8,9,2024-11-08,7:30p,Away,New York Knicks,L,94,116,2,7,L 1,19812,2:07,NaN
9,10,2024-11-10,3:30p,Home,Boston Celtics,L,107,113,2,8,L 2,17341,2:16,NaN
